In [3]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from sklearn.model_selection import train_test_split

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
data_path= '/content/drive/My Drive/Colab Notebooks/data/'
french_clean = pd.read_csv(data_path + 'french_clean.csv')
french_clean.drop(['Unnamed: 0'],axis=1,inplace=True)
french_clean.head()

,label,text,clean_text
0,0,La peinture me rend achy,le peinture me rendre achy
1,0,Les fans de nk ne sont-ils pas les meilleurs? ...,le fan de nk ne être il pas le meilleur je d...
2,1,Une autre belle journée. Coincé dans le bureau...,un autre bel journée coincer dans le bureau ...
3,0,": Dee, vous avez acheté une autre mini jupe? E...",dee vous avoir acheter un autre mini jup ...
4,1,Mon domestique à Hollywood a été éliminé! Trav...,mon domestique à hollywood avoir être éliminer...


In [ ]:
data_path= '/content/drive/My Drive/Colab Notebooks/data/'
allo_clean = pd.read_csv(data_path + 'allo_clean.csv')
allo_clean.head()

In [ ]:
allo_clean=allo_clean.rename(columns={"review": "text", "polarity": "label"})
allo_clean.drop(['Unnamed: 0','Unnamed: 0.1','film-url'],axis=1,inplace=True)
allo_clean=allo_clean[['label','text','clean_text']]
allo_clean.head()

In [ ]:
data_clean = pd.concat([allo_clean, french_clean], axis=0)
data_clean.head()

In [1]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop = set(stopwords.words('french'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [19]:
%timeit
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.utils.fixes import loguniform

X= data_clean['clean_text']
y= data_clean['label']

# Create Pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words=stop)),
    ('nb', MultinomialNB()),
])

# Set parameters to search
scoring={'AUC':'roc_auc', 'Accuracy':'accuracy','F1':'f1'

}
parameters = {
    'tfidf__ngram_range': ((1,1), (2,2), (1,2), (2,3), (3,3)),
    'tfidf__min_df':[0,0.1,0.2,0.3],
    'nb__alpha': (0.001, 0.01, 0.1, 1, 2,5,7,10),
}

# Perform random search
random_search = RandomizedSearchCV(pipeline, parameters, n_iter=200, n_jobs=-1, 
                           verbose=1, scoring = scoring, 
                           refit='AUC', cv=10, random_state=42)

random_search.fit(X,y)

NameError: ignored

In [28]:
random_search.best_score_

0.8565348588476255

In [29]:
random_search.best_params_

{'nb__alpha': 2, 'tfidf__min_df': 0, 'tfidf__ngram_range': (1, 2)}

In [ ]:
X_test = t
y_test = test['sentiment']
pred = random_search.best_estimator_.predict(X_test)

In [ ]:
baseline_score = pred
random_search.best_estimator_.score(X_test, y_test)

0.8711977186311787

In [31]:
import joblib
filename=f'NaiveBayes_alpha{2}_ngram{(1,2)}_min-df{0}_score-auc'
joblib.dump(pipeline,filename)

['NaiveBayes_alpha2_ngram(1, 2)_min-df0_score-auc']

In [ ]:
data_path= '/content/drive/My Drive/Colab Notebooks/data/'
test_french_tweets = pd.read_csv(data_path + 'labeled_french_tweets_1.csv')
test_french_tweets.head()

In [ ]:
clean_text('Bravo pour le trophie « ici il a pisse sur ton club »')